In [2]:
import os
import pandas as pd
import numpy as np
import copy
import csv

In [3]:
cn_dict = np.load("data/cn_dict.npy",allow_pickle='TRUE').item()
ad_dict = np.load("data/ad_dict.npy",allow_pickle='TRUE').item()
smci_dict = np.load("data/smci_dict.npy",allow_pickle='TRUE').item()
pmci_dict = np.load("data/pmci_dict.npy",allow_pickle='TRUE').item()

In [4]:
IMAGES = pd.read_csv("data/Big_cohort_MRI_Images_25Oct2024.csv")
IMAGES[0:10]

,image_id,subject_id,mri_visit,mri_date,mri_description,mri_type,mri_weighting,mri_sequence,mri_thickness,mri_te,...,mri_flip_angle,mri_acq_plane,mri_width,mri_height,mri_n_images,mri_pixel_x,mri_pixel_y,mri_mfr,mri_mfr_model,mri_field_str
0,70846,037_S_1421,sc,2007-08-27,"Head,3-P,2D,LOCALIZER",2D,T2,RM,5.0,1.600000,...,30,AXIAL,256.0,256.0,5,1.01563,1.01563,GE MEDICAL SYSTEMS,GENESIS_SIGNA,1.5
1,70847,037_S_1421,sc,2007-08-27,"Head,3-P,2D,LOCALIZER",2D,T2,RM,5.0,1.600000,...,30,CORONAL,256.0,256.0,5,1.01563,1.01563,GE MEDICAL SYSTEMS,GENESIS_SIGNA,1.5
2,70848,037_S_1421,sc,2007-08-27,"Head,3-P,2D,LOCALIZER",2D,T2,RM,5.0,1.600000,...,30,SAGITTAL,256.0,256.0,5,1.01563,1.01563,GE MEDICAL SYSTEMS,GENESIS_SIGNA,1.5
3,70849,037_S_1421,sc,2007-08-27,"Head,3-P,2D,LOCALIZER",2D,T2,RM,5.0,1.600000,...,30,AXIAL,256.0,256.0,5,1.01563,1.01563,GE MEDICAL SYSTEMS,GENESIS_SIGNA,1.5
4,70850,037_S_1421,sc,2007-08-27,"Head,3-P,2D,LOCALIZER",2D,T2,RM,5.0,1.600000,...,30,CORONAL,256.0,256.0,5,1.01563,1.01563,GE MEDICAL SYSTEMS,GENESIS_SIGNA,1.5
5,70851,037_S_1421,sc,2007-08-27,"Head,3-P,2D,LOCALIZER",2D,T2,RM,5.0,1.600000,...,30,SAGITTAL,256.0,256.0,5,1.01563,1.01563,GE MEDICAL SYSTEMS,GENESIS_SIGNA,1.5
6,70852,037_S_1421,sc,2007-08-27,FSE PD/T2,2D,PD,SE,3.0,10.184000,...,90,AXIAL,256.0,256.0,48,0.93750,0.93750,GE MEDICAL SYSTEMS,GENESIS_SIGNA,1.5
7,70853,037_S_1421,sc,2007-08-27,FSE PD/T2,2D,T2,SE,3.0,101.839996,...,90,AXIAL,256.0,256.0,48,0.93750,0.93750,GE MEDICAL SYSTEMS,GENESIS_SIGNA,1.5
8,70854,037_S_1421,sc,2007-08-27,MP-RAGE REPEAT,3D,T1,RM,1.2,4.092000,...,8,SAGITTAL,256.0,256.0,180,0.93750,0.93750,GE MEDICAL SYSTEMS,GENESIS_SIGNA,1.5
9,70855,037_S_1421,sc,2007-08-27,MP-RAGE,3D,T1,RM,1.2,4.092000,...,8,SAGITTAL,256.0,256.0,180,0.93750,0.93750,GE MEDICAL SYSTEMS,GENESIS_SIGNA,1.5


**Dict where patient id's are keys and the values are every single image with correspinding visit code.**

In [5]:
image_dict = {}

for index, row in IMAGES.iterrows():

    #Add filters to decide what images to include.
    if row['mri_type'] == '3D' and ('RAGE' in row['mri_description']) and ('REPEAT' not in row['mri_description']):

        if row['subject_id'] in image_dict:
            image_dict[row['subject_id']].append([row['mri_visit'], row['image_id']])
        else:
            image_dict[row['subject_id']] = [[row['mri_visit'], row['image_id']]]

In [17]:
#print(image_dict)

In [14]:
images_for_download = list()

for key in image_dict:
    for visit in image_dict[key]:
        images_for_download.append(visit[1])

print(len(images_for_download))

16096


Code for linking images to their respective paths

In [15]:
path = '/Users/olath/Documents/ADNI'
image_path_dict = {}


for subdir, dirs, files in os.walk(path):
    subdir_str = subdir.split('/')
    if len(subdir_str) == 9:
        for image in images_for_download:
            image_str = str(image)
            if ('I'+image_str) == (subdir_str[8]):
                image_path_dict[image] = subdir
                images_for_download.remove(image)

In [16]:
with open('images_with_paths.csv','w') as f:
    w = csv.writer(f)
    w.writerows(image_path_dict.items())

**Function for combining patient dict with image dict**

In [6]:
def combine_patients_pictures(in_dict):  

    dict_with_images = copy.deepcopy(in_dict)

    for patient in dict_with_images:

        if patient in image_dict:
            visits_pictures = image_dict[patient]

            for visit in dict_with_images[patient]:
                visit.append(list())

                for picture in visits_pictures: 

                    if visit[0] == picture[0]:
                        visit[3].append(picture[1])
        else:
            pass

    return dict_with_images


In [ ]:
cn_dict_with_images = combine_patients_pictures(cn_dict)
ad_dict_with_images = combine_patients_pictures(ad_dict)
smci_dict_with_images = combine_patients_pictures(smci_dict)
pmci_dict_with_images = combine_patients_pictures(pmci_dict)


print(cn_dict_with_images['011_S_0005'])
print(ad_dict_with_images['011_S_0003'])
print(smci_dict_with_images['022_S_0004'])
print(pmci_dict_with_images['023_S_0030'])

[['bl', '2005-09-30', 1.0], ['m06', '2006-03-09', 1.0], ['m12', '2006-09-06', 1.0], ['m24', '2007-09-07', 1.0], ['m36', '2008-09-10', 1.0]]
[['bl', '2005-09-30', 3.0], ['m06', '2006-03-13', 3.0], ['m12', '2006-09-13', 3.0], ['m24', '2007-09-12', 3.0]]
[['bl', '2005-11-08', 2.0], ['m06', '2006-05-02', 2.0], ['m12', '2006-11-14', 2.0], ['m18', '2007-05-14', 2.0], ['m36', '2008-11-18', 2.0]]
[['bl', '2005-10-20', 2.0]]


**Checking for any patient overlap between datasets**

In [ ]:
if set(cn_dict_with_images).intersection(ad_dict_with_images) or set(cn_dict_with_images).intersection(smci_dict_with_images) or set(cn_dict_with_images).intersection(pmci_dict_with_images) or set(ad_dict_with_images).intersection(smci_dict_with_images) or set(ad_dict_with_images).intersection(pmci_dict_with_images) or set(smci_dict_with_images).intersection(pmci_dict_with_images):
    print("Error. Patient overlap")

else:
    print("No patient overlap.")

    np.save("data/cn_dict_with_images.npy", cn_dict_with_images)
    np.save("data/ad_dict_with_images.npy", ad_dict_with_images) 
    np.save("data/smci_dict_with_images.npy", smci_dict_with_images) 
    np.save("data/pmci_dict_with_images.npy", pmci_dict_with_images)

No patient overlap.
